In [1]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(model="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

# a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there")

# a, b
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
chat = ChatOpenAI(temperature=0.1)

In [5]:
#message constructor
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert, And you only reply in Korean"),
    AIMessage(content="안녕하세요, 제 이름은 AIQ 입니다."),
    HumanMessage(content="What is the distance between Maxico and South Korea. Also, what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='멕시코와 한국 사이의 거리는 대략 12,000km입니다. 저는 AIQ입니다.')

In [8]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

prompt =template.format(country_a="Maxico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)



'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [13]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, And you only reply in {language}."),
    ("ai", "안녕하세요, 제 이름은 {name} 입니다."),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Maxico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    

p = CommaOutputParser()

p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [17]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do not reply anything else"),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [18]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']